本文目录

![Image Name](https://cdn.kesci.com/upload/image/qiejxoq701.png)


# 序言：关于A/B测试：

是一种源于生物医学中的双盲实验,将试验对象随机分组并针对不同组对象给不同的变量刺激，然后采集实验数据，运用统计学上的假设检验判断不同变量对实验的效果是否显著的科学实验方法。

![Image Name](https://cdn.kesci.com/upload/image/qiehzjoke1.jpg)图源：《增长高阶课》-曲卉

在互联网产品中主要用于通过对同一事物的不同变量 （界面变化，功能变化，人群变化，算法变化）进行测试来看用户更喜欢哪一种，从而达到提升数据指标的目的。

# 1.背景与思路

## 1.1分析背景

![Image Name](https://cdn.kesci.com/upload/image/qiei6xi5b6.jpg)图源：《增长高阶课》-曲卉

以上是做一个A/B测试实验的基本流程，本次分析的数据集来源于一家电商网站。希望通过对于一次AB测试数据的分析判断新旧两版页面在用户转化上是否有显著区别，帮助公司决定是应当采用新的页面，还是保留老的页面。
>（在整个A/B测试实验中属于第五步—分析实验结果）

## 1.2分析方法

In [1]:
'''【导入常用分析包】'''
#结构化数据分析工具集
import pandas as pd  
#科学计算基础软件包
import numpy as np
#图表绘制工具包
import matplotlib.pyplot as plt 
#基于 matplot, 导入 seaborn 会修改默认的 matplotlib 配色方案和绘图样式，这会提高图表的可读性和美观
import seaborn as sns
# 在 jupyter notebook 里面显示图表
%matplotlib inline

Duplicate key in file PosixPath('/Users/chengxin/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 758 ('font.family\t    : sans-serif')
Duplicate key in file PosixPath('/Users/chengxin/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 759 ('font.sans-serif\t    : SimHei')
Duplicate key in file PosixPath('/Users/chengxin/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 760 ('axes.unicode_minus   : False')


### 1.2.1查看源数据

In [2]:
#导入数据：ab_data
ABtest=pd.read_csv(r"../data/ab_data.csv")

In [3]:
#查看前5行数据
ABtest.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [4]:
#通过info函数查看数据信息
ABtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


### 1.2.2确定分析方法


![Image Name](https://cdn.kesci.com/upload/image/qieigu4sph.png)


首先我们要对数据进行检查和清洗，确保数据的准确性。
然后我们需要明确分析的问题，建立零假设和备选假设，判断样本的抽样分布类型和检测方式，最后根据得到的P值，判断实验性的显著性水平，得出结论。

# 2.数据分析

## 2.1数据清洗

查询并处理缺失值；查询并处理重复数据；检查并修改逻辑错误（在实验设计中给treatment组看到的是新页面，给control组对应的是旧的页面，不符合的数据需要删除）

In [5]:
'''【检查缺失值】'''
ABtest.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [6]:
'''【检查逻辑错误】'''
# 查询是否存在分组group与展示页面版本landing_page不符情况
t_old_error = len(ABtest[(ABtest['group'] == 'treatment') & (ABtest['landing_page'] == 'old_page')])
c_new_error = len(ABtest[(ABtest['group'] == 'control') & (ABtest['landing_page'] == 'new_page')])
print(
    '实验组看到老页面的人数:{}, 对照组看到新页面的人数:{}'.format(
        t_old_error,
        c_new_error))

实验组看到老页面的人数:1965, 对照组看到新页面的人数:1928


In [7]:
#正常数据数量：排除不匹配数据之后的行数
ABtest = ABtest[((ABtest['group'] == 'treatment') & (ABtest['landing_page'] == 'new_page'))
                      | ((ABtest['group'] == 'control') & (ABtest['landing_page'] == 'old_page'))]
print('正常数据',len(ABtest))

正常数据 290585


In [8]:
'''【检查重复值】'''
#drop_duplicates函数：查找重复值
ABtest.drop_duplicates(subset='user_id').head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [9]:
#删除重复值：仅保留最近一条信息
ABtest.drop_duplicates(subset=['user_id'],inplace=True,keep="first")
# 重置索引 
ABtest= ABtest.reset_index()
#通过info函数查看数据信息
ABtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290584 entries, 0 to 290583
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   index         290584 non-null  int64 
 1   user_id       290584 non-null  int64 
 2   timestamp     290584 non-null  object
 3   group         290584 non-null  object
 4   landing_page  290584 non-null  object
 5   converted     290584 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 13.3+ MB


In [10]:
#流量分配比例：看到新页面的用户占比多少
zb=ABtest[ABtest.landing_page=="new_page"].shape[0]/ABtest.shape[0]
print('看到新页面的用户占比：','%.2f%%'%(zb*100))

看到新页面的用户占比： 50.01%


看到新页面和老页面的用户各占50%左右，流量分配相对均衡

## 2.2假设检验

### 2.2.1明确问题

**【提出零假设和备选假设】**
> 记旧页面的转化率为 p1，新页面的转化率为 p2  

零假设  ：  p1>p2  即p1-p2>0  
备择假设  ：p1<p2  即 p1-p2<0  

**【确定抽样分布类型】**
本次实验满足的判断结果只有0和1（转化和未转化），符合0-1分布  
> **0-1分布**：就是n=1情况下的二项分布，任何只有两种结果的随机现象都服从0-1分布

**【确定检验类型】**

![Image Name](https://cdn.kesci.com/upload/image/qiej7fgj1a.png)

独立双样本，样本n>30，总体的均值和标准差未知，用Z检验。  
> 其中，pc为转化率的联合估计

**【确定检验方向】**
> 判断是否相等就是双侧,是否大于或小于就是单测  

根据备择假设，确定检验方向为单侧（左尾）

**【给定显著性水平α】**
本次检验α取0.05。

> **显著水平α**：时在进行假设检验时先确定的一个可允许的作为判断界限的小概率标准（用于和P值比较大小而最终确定结论，一般常用值：0.05，0.01）

### 2.2.2计算检验统计量

Z=（p1-p2）/sqrt[p1×(1-p1)/n1+p2×(1-p2)/n2] 

In [11]:
# 旧版、新版用户数（query是条件过滤函数）
n_old = ABtest.query('group=="control"').shape[0]
n_new = ABtest.query('group=="treatment"').shape[0]
# 旧版、新版转化用户数
convert_old = ABtest.query('group=="control" & converted==1').shape[0]
convert_new = ABtest.query('group=="treatment" & converted==1').shape[0]
# 旧版、新版转化率
p_old = convert_old / n_old
p_new = convert_new / n_new
p_c = (convert_old + convert_new)/(n_old + n_new)
z = (p_old - p_new)/ np.sqrt(p_c*(1 - p_c)*( 1/n_old + 1/n_new))
from scipy.stats import norm
z_alpha = norm.ppf(0.05)
# 若为双侧，则norm.ppf(0.05/2)
print('旧版总受试用户数:', n_old, '旧版转化用户数:', convert_old, '旧版转化率:', p_old)
print('新版总受试用户数:', n_new, '新版转化用户数:', convert_new, '新版转化率:', p_new)
print('转化率的联合估计:', p_c)
print('检验统计量z:', z)
print(z_alpha)
result = "落入拒绝域，拒绝零假设" if abs(z) > abs(z_alpha) else "接受零假设"
print(result)

旧版总受试用户数: 145274 旧版转化用户数: 17489 旧版转化率: 0.1203863045004612
新版总受试用户数: 145310 新版转化用户数: 17872 新版转化率: 0.12299222352212512
转化率的联合估计: 0.12168942543292129
检验统计量z: -2.1484056695589
-1.6448536269514729
落入拒绝域，拒绝零假设


In [12]:
# 合并标准差
std_old = ABtest[ABtest.landing_page=="old_page"].converted.std()
std_new = ABtest[ABtest.landing_page=="new_page"].converted.std()
s = np.sqrt(((n_old - 1)* std_old**2 + (n_new - 1)* std_new**2 ) / (n_old + n_new - 2))
# 效应量Cohen's d
d = (p_old - p_new) / s
print('Cohen\'s d为：', d)

Cohen's d为： -0.007970992391336005


# 3.分析结论

Cohen's d的值约为-0.008304，绝对值很小。两者虽有显著性水平5%时统计意义上的显著差异，但差异的效应量很小。可以理解为**显著有差异，但差异的大小不显著**。